In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import json

from matplotlib import  pyplot as plt
import seaborn as sns
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr import LotrExperiment

from skimage.filters import threshold_otsu

In [3]:
plt.close("all")
master =  Path(r"Z:\Hagar\old data e0040_v13 with heading")
fish_list = list(master.glob("*_f*"))

In [4]:
num_fish = len(fish_list)
print(num_fish)

3


In [5]:
fig, ax = plt.subplots(2,1, figsize=(5,4), sharex=True)
plt.subplots_adjust(hspace=0.4, bottom=0.2)
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)

ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

ax[1].set_xlabel('Reliability of tuning to visual motion')
ax[0].set_ylabel('# Neurons')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '# Neurons')

In [6]:
for fish in range(num_fish):
    path = fish_list[0]
    
    selected = fl.load(path / "selected.h5")
    rel_rois = fl.load(path / "reliable_rois.h5")['reliability_arr']
    in_ahb = fl.load(path / "mask_and_idxs_ahb.h5")['cells_idx']
    
    rel_hdns = rel_rois[selected]
    rel_rois = rel_rois[in_ahb]
    rand_rois = np.random.rand(np.shape(rel_rois)[0])
    rand_hdns = np.random.rand(np.shape(rel_hdns)[0])
    
    ax[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
    ax[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
    

    ax[1].scatter(rel_rois, rand_rois, color='skyblue', s=5)
    ax[1].scatter(rel_hdns, rand_hdns, color='forestgreen', s=5)


In [7]:
ax[0].legend(prop={'size': 10})

In [ ]:
fig.savefig(master / "hdns vs rel index in ahb.jpg", dpi=300)

In [10]:
fig2, ax2 = plt.subplots(1, 3, figsize=(7,2.5), sharex=True)

for i in range(3):
    ax2[i].spines['top'].set_visible(False)
    ax2[i].spines['right'].set_visible(False)

    ax2[i].set_xlabel('Reliability of tuning to visual motion')


ax2[1].set_ylabel('Max negative correlation')
ax2[2].set_ylabel('Max negative correlation')
ax2[0].set_xlim([-0.2, 1])
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.9, wspace=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
n_neurons = np.zeros((num_fish, 2)) # n_hdns, n_vis
for fish in range(num_fish):
    print(fish)
    path = fish_list[fish]
    
    selected = fl.load(path / "selected.h5")
    rel_rois = fl.load(path / "reliable_rois.h5")['reliability_arr']
    in_ahb = fl.load(path / "mask_and_idxs_ahb.h5")['cells_idx']
    
    rel_hdns = rel_rois[selected]
    rel_rois = rel_rois[in_ahb]
    
    n_neurons[fish, 0] = np.shape(selected)[0]
    
    traces = fl.load(path / "filtered_traces.h5", "/detr")
    exp = LotrExperiment(path)
    
    t_start_s = 50
    t_end = exp.n_pts//2
    t_lims = (t_start_s*exp.fn, t_end)
    t_slice = slice(*t_lims)
    cc_wnd = 3600
    i_array = np.arange(t_slice.start, t_slice.stop, cc_wnd*exp.fn)
    cc_mats = np.zeros((traces.shape[1], traces.shape[1], len(i_array)))

    for n, i in enumerate(i_array):
        cc_mats[:, :, n] = np.corrcoef(traces[i:i + cc_wnd*exp.fn, :].T)
    corr_mat = np.nanmean(cc_mats, 2)
    
    neg_corr = np.nanmin(corr_mat, axis=0)

    neg_corr_hdns = neg_corr[selected]
    neg_corr_ahb = neg_corr[in_ahb]
    
    ### find threshold with otsus:
    thresh = threshold_otsu(rel_rois)
    print(thresh)
    selected_vis = np.where(rel_rois > thresh)[0]
    n_neurons[fish, 1] = np.shape(selected_vis)[0]
    
    rel_vis = rel_rois[selected_vis]
    neg_corr_vis =  neg_corr_ahb[selected_vis]
    
    ax2[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
    ax2[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
    ax2[0].set_ylabel('# Neurons')
    #ax2[0].legend(prop={'size': 10})


    #ax2[1].scatter(rel_rois, neg_corr_ahb, color='skyblue', s=5, label='All ROIs')
    ax2[1].scatter(rel_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
    ax2[1].scatter(rel_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')
    
    if fish == 0:
        ax2[2].scatter(rel_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
        ax2[2].scatter(rel_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')
        

0
0.2991564803387107
1
0.37537614599938557
2
0.22782038751028447


In [12]:
plt.subplots_adjust(left=0.05, bottom=0.2, right=0.95, wspace=0.4)

In [ ]:
fig2.savefig(master / "hdns vs rel index scatter2d with all neurons otsus thresh.jpg", dpi=300)
fig2.savefig(master / "hdns vs rel index scatter2d with all neurons otsus thresh.pdf", dpi=300)

In [13]:
fig3, ax3 = plt.subplots(1, 2, figsize=(7,4), sharex=True)

ax3[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
ax3[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
ax3[0].set_ylabel('# Neurons')
ax3[0].legend(prop={'size': 10})

ax3[0].spines['top'].set_visible(False)
ax3[0].spines['right'].set_visible(False)

ax3[1].scatter(rel_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
ax3[1].scatter(rel_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')

ax3[1].legend(prop={'size': 10})
ax3[1].set_ylim([-1, 0])
ax3[1].set_xlim([-0.3, 1])
ax3[0].set_xlim([-0.3, 1])

ax3[1].spines['top'].set_visible(False)
ax3[1].spines['right'].set_visible(False)

ax3[1].set_xlabel('Reliability of tuning to visual motion')
ax3[0].set_xlabel('Reliability of tuning to visual motion')
ax3[1].set_ylabel('Max negative correlation')
plt.subplots_adjust(left=0.1, bottom=0.2, wspace=0.3, right=0.95)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig3.savefig(path / "hdns vs rel index scatter2d.jpg", dpi=300)
fig3.savefig(path / "hdns vs rel index scatter2d.pdf", dpi=300)

In [27]:
n_neurons_mean = np.nanmean(n_neurons, 0) //1
n_neurons_sem = np.nanstd(n_neurons, 0) // num_fish

In [28]:
print('Number of HDNs (mean±sem) = ' + str(n_neurons_mean[0]) + '±' + str(n_neurons_sem[0]))
print('Number of visually tuned (mean±sem) = ' + str(n_neurons_mean[1]) + '±' + str(n_neurons_sem[1]))

Number of HDNs (mean±sem) = 127.0±13.0
Number of visually tuned (mean±sem) = 219.0±42.0
